In [1]:
!gdown --id "1jBWSOLNo-dwIRjuBj7AcyJWJQHVbFP0M"

Downloading...
From: https://drive.google.com/uc?id=1jBWSOLNo-dwIRjuBj7AcyJWJQHVbFP0M
To: /content/Dataset.zip
63.3MB [00:00, 103MB/s] 


In [2]:
!unzip Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/0/0926.jpg      
  inflating: __MACOSX/Dataset/0/._0926.jpg  
  inflating: Dataset/0/1437.jpg      
  inflating: __MACOSX/Dataset/0/._1437.jpg  
  inflating: Dataset/0/0729.jpg      
  inflating: __MACOSX/Dataset/0/._0729.jpg  
  inflating: Dataset/0/1351.jpg      
  inflating: __MACOSX/Dataset/0/._1351.jpg  
  inflating: Dataset/0/1345.jpg      
  inflating: __MACOSX/Dataset/0/._1345.jpg  
  inflating: Dataset/0/1423.jpg      
  inflating: __MACOSX/Dataset/0/._1423.jpg  
  inflating: Dataset/0/0715.jpg      
  inflating: __MACOSX/Dataset/0/._0715.jpg  
  inflating: Dataset/0/0073.jpg      
  inflating: __MACOSX/Dataset/0/._0073.jpg  
  inflating: Dataset/0/0067.jpg      
  inflating: __MACOSX/Dataset/0/._0067.jpg  
  inflating: Dataset/0/1379.jpg      
  inflating: __MACOSX/Dataset/0/._1379.jpg  
  inflating: Dataset/0/0701.jpg      
  inflating: __MACOSX/Dataset/0/._0701.jpg  
  inflating: Dataset/0/0739.jpg     

In [3]:
import cv2
import numpy as np
import imutils
import tensorflow_addons as tfa
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications import InceptionResNetV2
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

IMG_WIDTH, IMG_HEIGHT = 620, 480
EPOCHS = 10
BATCH_SIZE= 10
n_classes = 3

In [4]:
def find_ROI(img):
    STRUCTURING_ELEMENT_12 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 20))
    STRUCTURING_ELEMENT_23 = cv2.getStructuringElement(cv2.MORPH_RECT, (60, 1))
    STRUCTURING_ELEMENT_24 = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))

    STRUCTURING_ELEMENT_3 = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
    STRUCTURING_ELEMENT_31 = cv2.getStructuringElement(cv2.MORPH_RECT, (150, 5))

    img = img.astype(np.uint8)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    gray = cv2.resize(gray, (IMG_WIDTH - 200, IMG_HEIGHT - 200))

    blurred_image = cv2.GaussianBlur(gray, (5, 5), 0)

    gray = cv2.bilateralFilter(blurred_image, 13, 15, 15)

    adther = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 2)

    canny = cv2.Canny(adther, 10, 240)

    lines = cv2.HoughLinesP(canny, 1, np.pi/180, 1)
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(canny, (x1, y1), (x2, y2), (255, 255, 255), 3)

    openningV = cv2.morphologyEx(canny.copy(), cv2.MORPH_OPEN, STRUCTURING_ELEMENT_12)

    openningV = cv2.copyMakeBorder(openningV, 200, 200, 200, 200, cv2.BORDER_CONSTANT, 0)

    closeH = cv2.morphologyEx(openningV, cv2.MORPH_CLOSE, STRUCTURING_ELEMENT_23)

    openH = cv2.morphologyEx(closeH, cv2.MORPH_OPEN, STRUCTURING_ELEMENT_24)

    dilate = cv2.dilate(openH, STRUCTURING_ELEMENT_3)
    closeHAgain = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, STRUCTURING_ELEMENT_31)

    closeHAgain = closeHAgain[100:-100, 100:-100]
    return closeHAgain

In [5]:
def find_condidate_pelak(img):
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    res = find_ROI(img)

    new_image = cv2.bitwise_and(img, img, mask=res)

    # contours = cv2.findContours(res.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # contours = imutils.grab_contours(contours)
    #
    # for contour in contours:
    #
    #     # Draw line around each contour.
    #     cv2.drawContours(new_image, [contour], -1, (255, 255, 255), 2)
    #
    #     # Write the contour shape and its color in the center of image.
    return new_image

In [6]:
def my_preprocessing_function(img):
    # cv2.imshow('img', img)
    # cv2.waitKey(0)
    new_image = find_condidate_pelak(img)

    new_image = new_image.astype(np.float32)
    # cv2.imshow('new_image', new_image)
    # cv2.waitKey(0)
    final_image = preprocess_input(new_image)
    # cv2.imshow('final_image', final_image)
    # cv2.waitKey(0)

    return final_image

In [7]:
def build_model_with_InceptionResNetV2():
    model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    for layer in model.layers:
        layer.trainable = True
    avgPooling = GlobalAveragePooling2D()(model.output)
    dense1 = Dense(1024, activation='relu')(avgPooling)
    dense2 = Dense(n_classes, activation='softmax')(dense1)
    model = Model(model.input, dense2)
    return model

In [8]:
train_datagen_aug = ImageDataGenerator(width_shift_range=0.2, height_shift_range=0.1,
                                       rotation_range=30, shear_range=0.1, zoom_range=0.1,
                                       horizontal_flip=True,
                                       preprocessing_function=my_preprocessing_function,
                                       validation_split=0.2)

In [9]:
train_generator_aug = train_datagen_aug.flow_from_directory('/content/Dataset', target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, shuffle=True, subset='training')

validation_generator_aug = train_datagen_aug.flow_from_directory('/content/Dataset', target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=BATCH_SIZE, shuffle=True, subset='validation')

Found 2183 images belonging to 3 classes.
Found 544 images belonging to 3 classes.


In [10]:
model = build_model_with_InceptionResNetV2()
loss = 'categorical_crossentropy'
optimizer = Adam(lr=1e-4)
F1_metric = tfa.metrics.F1Score(num_classes=3)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', F1_metric])
# model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy', tfa.metrics.F1Score(num_classes=3, average="micro", threshold = threshold)])
early_stopping = EarlyStopping(monitor='val_accuracy', patience=EPOCHS, restore_best_weights=True)
model.fit(train_generator_aug, validation_data=validation_generator_aug, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping])

219062272/219055592 [==============================] - 1s 0us/step
Epoch 1/10
219/219 [==============================] - 429s 2s/step - loss: 0.6828 - accuracy: 0.7244 - f1_score: 0.4648 - val_loss: 0.4734 - val_accuracy: 0.8051 - val_f1_score: 0.5361
Epoch 2/10
219/219 [==============================] - 400s 2s/step - loss: 0.4337 - accuracy: 0.8227 - f1_score: 0.6041 - val_loss: 0.3775 - val_accuracy: 0.8566 - val_f1_score: 0.7162
Epoch 3/10
219/219 [==============================] - 397s 2s/step - loss: 0.3783 - accuracy: 0.8594 - f1_score: 0.6815 - val_loss: 0.4210 - val_accuracy: 0.8621 - val_f1_score: 0.6054
Epoch 4/10
219/219 [==============================] - 403s 2s/step - loss: 0.3466 - accuracy: 0.8613 - f1_score: 0.6985 - val_loss: 0.3633 - val_accuracy: 0.8640 - val_f1_score: 0.7091
Epoch 5/10
219/219 [==============================] - 398s 2s/step - loss: 0.3018 - accuracy: 0.8873 - f1_score: 0.7937 - val_loss: 0.4478 - val_accuracy: 0.8401 - val_f1_score: 0.6238
Epoch 6/

In [11]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
%cd gdrive/My Drive/ComputerVision/FinalProject

/content/gdrive/My Drive/ComputerVision/FinalProject


In [13]:
model.save('InceptionResNetV2_with_shift_5.h5')

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd gdrive/My Drive/ComputerVision/FinalProject

/content/gdrive/My Drive/ComputerVision/FinalProject


In [ ]:
img = cv2.imread('Dataset/1/0002.jpg')

fianl_image = my_preprocessing_function(img)
fianl_image = np.expand_dims(fianl_image, axis=0)
# fianl_image = cv2.resize(img, (620, 480))


model.predict(fianl_image)
# model.summary()

(480, 620, 3)
(1, 480, 620, 3)


array([[0.707049  , 0.12918511, 0.16376586]], dtype=float32)